In [1]:
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re
import os
import sys
import glob
import shutil
from pathlib import Path

In [2]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='C:\chromedriver', options=options)
driver.implicitly_wait(0)

In [3]:
# Mở trang deal hot sale của Tiki
url = "https://tiki.vn/deal-hot?tab=now"
driver.get(url)
driver.maximize_window()

In [4]:
# Đợi 20s cho trang load
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-view-id="product_container"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup1=BeautifulSoup(html_of_interest, 'lxml')

In [5]:
sleep(2)
num = 0
# List mục sale 
a_list = []
# Tạo 1 list chứa tất cả link
links = []
# Tìm và cho vào list vị trí của các mục deal sale
muc_sale = driver.find_elements_by_css_selector('div[class="styles__TagItemWrapper-wpma16-1 cjFfzU"]')
muc_sale.append("end")
for i in muc_sale:
    sleep(2)
    num+=1
    # Kéo chuột xuống cuối trang để hiện thêm deal sale
    while True:
        try:
            sleep(4)
            ele = driver.find_element_by_css_selector('div[class="styles__LoaderWrapper-sc-1466pn8-1 jIlCuH"]')
            ele.location_once_scrolled_into_view
        except:
            print("Xong một mục")
            break

    ## Lấy link tất cả các sản phẩm trong mục

    # List vị trí các sản phẩm
    list_item = driver.find_elements_by_css_selector('div[class="styles__Wrapper-sc-6jfdyd-0 bruqdn"]')

    # Loop từng vị trí
    for l in list_item:
        # Lọc lấy link
        link = l.get_attribute('innerHTML').split('"')[1]
        # Cho vào list links đã tạo
        if link not in links:
            links.append(link)

    # Khi quét xong mục cuối thì kết thúc
    if i =="end":
        break
            
    # Kéo lên đầu trang để chuẩn bị qua mục deal khác
    driver.execute_script("window.scroll(0, 0);")
    sleep(1)

    # Nhấn nút chuyển để hiện thêm mục deal
    if num%10==0:
        try:
            button = driver.find_element_by_css_selector('a[class="slick-arrow slick-next"]')
            driver.execute_script("arguments[0].click();", button)
            sleep(3)
        except:
            break
    ## Kiểm tra xem mục đã được lấy link chưa
    if i not in a_list:
        # Nếu chưa thì nhấn vào để lấy link
        a_list.append(i)
        print(i.text)
        i.click()

Xong một mục
Thương Hiệu Nổi Bật
Xong một mục
Freeship +
Xong một mục
Dưới 49K
Xong một mục
Siêu Sale Shop Mới
Xong một mục
Thời Trang & Làm Đẹp
Xong một mục
Phong Cách Sống
Xong một mục
Bách Hóa
Xong một mục
Công Nghệ
Xong một mục
Nhà Sách Tiki
Xong một mục
Hàng Quốc Tế
Xong một mục


In [ ]:
df_item = []
error_link = []

for link in links:   
    
    # Mở link
    driver.get(link)
    
    # Đợi vài giây để load link
    element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="__next"]')))
    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')
    sleep(2)
    
    # Tạo dict chứa thông tin của sản phẩm
    dict_item = {}
    
    # Tên sản phẩm
    dict_item["Tên sản phẩm"] = soup.select('h1[class="title"]')[0].text
    
    # Check xem link đã hết sale chưa
    try:
        # Chưa hết thì lấy thông tin
        # % sale
        dict_item["% Sale"] = driver.find_element_by_class_name('flash-sale-price').text.split('\n')[1].split('%')[0] + "%"

        # Giá gốc
        dict_item["Giá gốc"] = driver.find_element_by_class_name('flash-sale-price').text.split('\n')[1].split('%')[1]

        # Giá đã giảm
        dict_item["Giá đã giảm"] = driver.find_element_by_class_name('flash-sale-price').text.split('\n')[0]
    except:
        # Hết sale thì cho vào list link đã hết sale
        error_link.append(link)
        continue
        
    # Các lựa chọn màu
    color_list = soup.select('span[class="option-label"]')
    if color_list != []:
        dict_item["Màu sắc"] = ", ".join([color.text for color in color_list])
    else:
        dict_item["Màu sắc"] = "Chỉ có 1 màu"
        
    # Size
    size_list = soup.select('button[data-view-id="pdp_main_select_configuration_item"]')
    if size_list != []:
        dict_item["Kích cỡ"] = ", ".join([size.text for size in size_list])
    else:
        dict_item["Kích cỡ"] = "Chỉ có 1 kích cỡ"
    
    # Ảnh sản phẩm
    dict_item["Ảnh sản phẩm"] = soup.select('div[data-view-id="pdp_main_view_gallery"]')[0].img.attrs['src']
        
    # Số lượng đã bán
    sl_da_ban = soup.select('div[class="indexstyle__QuantitySold-qd1z2k-5 cuXUxf"]')
    if sl_da_ban != []:
        dict_item["Số lượng đã bán"] = sl_da_ban[0].text
    else:
        dict_item["Số lượng đã bán"] = "0"

    # Đánh giá sao
    danh_gia_sao = eval(driver.find_elements_by_css_selector('script[type="application/ld+json"]')[2].get_attribute('innerHTML'))
    if "ratingValue" in danh_gia_sao:
        dict_item["Đánh giá sao"] = danh_gia_sao["ratingValue"]
    else:
        dict_item["Đánh giá sao"] = "Chưa có đánh giá"
    
    # Số lượt đánh giá
    luot_danh_gia = soup.select('a[data-view-id="pdp_main_view_review"]')
    if luot_danh_gia != []:
        dict_item["Số lượt đánh giá"] = int(''.join(luot for luot in luot_danh_gia[0].text if luot.isdigit()))
    else:
        dict_item["Số lượt đánh giá"] = "Chưa có đánh giá"
    
    # Link sản phẩm
    dict_item["Link"] = link
    
    df_item.append(dict_item)
    sleep(13)

df = pd.DataFrame(df_item)
df

In [8]:
len(links)

4677